This notebook contains the code that was used in the research to calculate DTW, perform and plot clustering, see metrics.

The dataset used here may be found [on Google Drive](https://drive.google.com/drive/folders/10-Blxv1K0X57u85mXJTX5R1NuRB1qWct?usp=sharing).
The code largely depends on the data but one may use basic fucntions and approaches from this notebook to analyse their own data.

NB! Due to Google Colab resource restrictions and/or imcompatibily of some libraries, once we received any however big matrix, we could work with it only once before the kernel crashed. Be careful choosing how to run this code and try to save as much data locally/on colab's drive as you can so when a crash happens, your data isn't lost.

Contact me if you have any problems running or understanfing the code (tg: iphilatov)

## Preliminaries

In [ ]:
!pip install librosa==0.9.2
!pip install scikit-bio

In [ ]:
from matplotlib.patches import ConnectionPatch
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import numpy as np
import re
import random
import scipy.spatial.distance as dist
from scipy.cluster.hierarchy import dendrogram, linkage, median, maxdists, fcluster
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from scipy.spatial.distance import squareform
from sklearn.metrics.pairwise import haversine_distances
from scipy.stats import pearsonr
from skbio.stats.distance import mantel
from scipy.io import wavfile
from math import radians
import IPython.display as ipyd
import librosa
import os
import librosa.display
import pandas as pd

In [ ]:
# this piece of code is needed when you use data from google drive
from google.colab import drive
drive.mount('/content/drive')

## Functions for DTW

In [ ]:
def dp(dist_mat):

    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)

In [ ]:
# this code compares two audio files by DTW

def two_comparison(a_audio, b_audio):
  f_sx, x = wavfile.read(a_audio)
  f_sy, y = wavfile.read(b_audio)

  n_fft = int(0.025*f_s)      # 25 ms
  hop_length = int(0.01*f_s)  # 10 ms
  
  mel_spec_x = librosa.feature.melspectrogram(
    x/1.0, sr=f_sx, n_mels=40,
    n_fft=n_fft, hop_length=hop_length
    )
  log_mel_spec_x = np.log(mel_spec_x)

  mel_spec_y = librosa.feature.melspectrogram(
    y/1.0, sr=f_sy, n_mels=40,
    n_fft=n_fft, hop_length=hop_length
    )
  log_mel_spec_y = np.log(mel_spec_y)

  x_seq = log_mel_spec_x.T
  y_seq = log_mel_spec_y.T

  dist_mat = dist.cdist(x_seq, y_seq, "cosine")
  path, cost_mat = dp(dist_mat)
  print("Alignment cost: {:.4f}".format(cost_mat[-1, -1]))

  M = y_seq.shape[0]
  N = x_seq.shape[0]
  print(
        "Normalized alignment cost: {:.8f}".format(
        cost_mat[-1, -1]/(M + N))
        )

In [ ]:
# this code compares multiple files by DTW

def comparison(list_of_audio):
  
  q = len(list_of_audio)
  seq_dict = dict()
  not_df = dict()

  for audio in list_of_audio:
    
    audio_name = str(audio)

    f_s, z = wavfile.read(audio)
    
    n_fft = int(0.025*f_s)      # 25 ms
    hop_length = int(0.01*f_s)  # 10 ms

    mel_spec_z = librosa.feature.melspectrogram(
    z/1.0, sr=f_s, n_mels=40,
    n_fft=n_fft, hop_length=hop_length
    )
  
    log_mel_spec_z = np.log(mel_spec_z)

    z_seq = log_mel_spec_z.T

    seq_dict[audio_name] = z_seq

  for name_main, seq_main in seq_dict.items():
    
    row_dict = dict()

    M = seq_main.shape[0]

    for name_snd, seq_snd in seq_dict.items():

      dist_mat = dist.cdist(seq_main, seq_snd, "cosine")
      path, cost_mat = dp(dist_mat)

      N = seq_snd.shape[0]

      normalized = cost_mat[-1, -1]/(M + N)

      row_dict[name_snd] =  "%.8f" % normalized
    
    not_df[name_main] = row_dict

  return pd.DataFrame(not_df)

In [ ]:
# this function may be handy if one wants to see isc visualization of an audio
def osc_visual(audio_file):
  y, sr = librosa.load(audio_file)
  librosa.display.waveshow(y, sr=sr) 

# Original clustering



In [ ]:
# this is the most basic clustering and its visualisations
#start here if you just need some cluster analysis and a dendrogram

def clustering_alpha(df, draw = True, fs = (20,15)):

  # make an condensed matrix from an uncondensed one 
  condense = squareform(df, checks=False)
  # you may want to try this line instead of the one above if it doesn't work
  #condense = squareform(df) 

  # getting labels for the plot (not the best way to use pandas though)
  labels = list(df.columns)

  # calculating clusters here (needs lots of resource)
  H = linkage(condense, 'ward')

  # make a dendrogram
  if draw == True:
    fig = plt.figure(figsize=fs)
    dn = dendrogram(H, labels = labels, orientation='right')
    plt.show()

# Creating data to use for clustering

## Data preprocessing

In [ ]:
# reading names of files here  

first_audio_names = [file for file in os.listdir('/content/drive/MyDrive/output')]
len(first_audio_names)

In [ ]:
# here we check that all audio fit our name pattern; if some don't, we'll work with them later

count = 0

for file in first_audio_names:
    if re.fullmatch("\d+_(bylo|eto|menja|oni|tozhe)_[A-Za-z]+\d*_\w+.wav", file):
        count += 1
    else:
        print(file) 

print(count)

In [ ]:
# getting rid of extension info in file names

all_audio_names = list()

for name in first_audio_names:
  if name[-4:] == '.wav':
    name = name[:-4]    
  all_audio_names.append(name)

In [ ]:
# this function lets you retrive certain subsets from the dataset; you can pick files by a speaker, a word and a corpus 

def corpora_search(data_list, word = None, speaker = None, corpus = None):

  if word in ['bylo', 'eto', 'oni', 'tozhe']:
    by_word_list = [name for name in data_list if re.fullmatch("\d+_{}_[A-Za-z]+\d*_\w+".format(word), name)]
  # "меня" has some exceptions so you nay want to process them in a different way
  elif word == 'menja':
    by_word_list  = [name for name in data_list if re.fullmatch("\d+_menja_([A-Za-z]+@)?[A-Za-z]+\d*_\w+", name)]
  elif word == None:
    by_word_list = data_list
  else:
    print('Error: wrong word choice')

  if speaker == None:
    by_speaker_list = by_word_list
  else:
    by_speaker_list = [name for name in by_word_list if re.fullmatch("\d+_[A-Za-z]+_{}_\w+".format(speaker), name)]

  if corpus == None:
    by_corpus_list = by_speaker_list
  else:
    by_corpus_list = [name for name in by_speaker_list if re.fullmatch("\d+_[A-Za-z]+_([A-Za-z]+@)?[A-Za-z]+\d*_{}".format(corpus), name)]

  return by_corpus_list

In [ ]:
# creating a subset of data which contains examples of the pronunciation of the word 'тоже'

the_word = 'tozhe'
only_one_word = corpora_search(all_audio_names, word = the_word)

## Binding meta data from corpus with the dataset

In [ ]:
# giving each corpus its own colour; we practically do not need a file with meta data here

list_of_corpora = list() 
for name in all_audio_names:
  corpora_name = name.split('_')[-1].split('.')[0]
  if corpora_name not in list_of_corpora and corpora_name[-1] != ')':
    list_of_corpora.append(corpora_name)

colours = ['black', 'green', 'red', 'cyan', 'crimson', 'yellow', 'blue', 'rosybrown', 'magenta', 'brown', 'olive', 'teal', 'orangered', 'purple', 'indigo']
len(colours)

color4cor = dict(zip(list_of_corpora, colours))

color_by_corpus = dict()

for name in all_audio_names:
  place = name.split('_')[-1].split('.')[0]
  colour = color4cor[place]
  color_by_corpus[name] = colour

In [ ]:
# reading meta data

xlsx_path = '/content/drive/MyDrive/meta.xlsx'
meta_df = pd.read_excel(xlsx_path)

In [ ]:
# giving each gender its colour - we have only males and females, so the colours user are blue (males) and red (females)

color_by_gender = dict()

for name in all_audio_names:
    
    person = name.split('_')[2]

    try:
        sex = meta_df.loc[meta_df.string_id == person].gender.item()
    except ValueError:
        continue
    
    if sex.lower() == 'f':
        color = 'red'
    if sex.lower() == 'm':
        color = 'blue'

    color_by_gender[name] = color

# working with exceptions 
for name in all_audio_names:
    if name.split('_')[2] == 'MNS1916' or name.split('_')[2] == 'tx@MLI1941':
        color_by_gender[name] = 'red'

In [ ]:
# creating a colour grade and giving each file its colour on it based on speaker's age

color_by_age = dict()
cmap = cm.get_cmap('viridis')

for name in all_audio_names:
    
    person = name.split('_')[2]

    # the research was made in 2023
    try:
        age = 2023 - int(meta_df.loc[meta_df.string_id == person].year_of_birth.item())
    except ValueError:
        continue

    color = cmap(1-age/100)

    color_by_age[name] = color

# working with exceptions 
for name in all_audio_names:
    if name.split('_')[2] == 'MNS1916':
        color_by_age[name] = cmap((2023-1916)/100)
    elif name.split('_')[2] == 'tx@MLI1941':
        color_by_age[name] = cmap((2023-1941)/100)

In [ ]:
# this code creates a dictionary where each file is assigned to its corpus; may be useful later

file_corpora_dict = dict()

for audio in all_audio_names:
  for corpora in list_of_corpora:
    if corpora in audio:
      file_corpora_dict[audio] = corpora

## Calculating a matrix

In [ ]:
# this is a modified version of `comparison` function which works better with research's dataset

def comparison4output(list_of_audio):

  q = len(list_of_audio)
  seq_dict = dict()
  not_df = dict()

  for audio in list_of_audio:
    
    audio_path = '/content/drive/MyDrive/output/' + audio + '.wav'

    audio_name = str(audio)

    f_s, z = wavfile.read(audio_path)
    
    n_fft = int(0.025*f_s)      # 25 ms
    hop_length = int(0.01*f_s)  # 10 ms

    mel_spec_z = librosa.feature.melspectrogram(
    z/1.0, sr=f_s, n_mels=40,
    n_fft=n_fft, hop_length=hop_length
    )
  
    log_mel_spec_z = np.log(mel_spec_z)

    z_seq = log_mel_spec_z.T

    seq_dict[audio_name] = z_seq

  for name_main, seq_main in seq_dict.items():
    
    row_dict = dict()

    M = seq_main.shape[0]

    for name_snd, seq_snd in seq_dict.items():

      dist_mat = dist.cdist(seq_main, seq_snd, "cosine")
      path, cost_mat = dp(dist_mat)

      N = seq_snd.shape[0]

      normalized = cost_mat[-1, -1]/(M + N)

      row_dict[name_snd] =  "%.8f" % normalized
    
    not_df[name_main] = row_dict

  return pd.DataFrame(not_df)

In [ ]:
# calculating a matrix
# calculation may take pretty long

matrix = comparison4output(only_one_word)

In [ ]:
# it is recommended to save the matrix as Colab is bad at handling lots of massive so a kernel may crash
matrix.to_pickle('orig.pkl')

# Many clusterings
This section present a couple of approaches to work with clustering that were used in our research 

## Clustering with meta data

In [ ]:
# this function is based on the original clustering function
# it uses meta data to colour labels 

def clustering_beta(df, coloring, draw=True, fs=(20, 15)):
    labels = [str(label) for label in df.columns]
    condense = squareform(df, checks=False)
    H = linkage(condense, 'ward')

    # creating the dendrogram and set the color of the leaves
    if draw:
        fig = plt.figure(figsize=fs)
        dn = dendrogram(H, labels=labels, orientation='left', color_threshold = 0)

        ax = plt.gca()
        ax.invert_yaxis()
        xlbls = ax.get_ymajorticklabels()
        for lbl in xlbls:
            color_decider = lbl.get_text()
            lbl.set_color(coloring[color_decider])

        plt.show()

In [ ]:
clustering_beta(matrix, color_by_corpus)

## Clustering and plotting independently

In [ ]:
# these two functions do basically the same as the previous one but they are separated
# so labels and linkage matrix don't have to calculated again to plot a dendrogram with different meta dat

def clustering_delta(df):
  condense = squareform(df, checks=False)
  labels = list(df.columns)
  H = linkage(condense, 'ward')
  return H, labels

def drawing_clustering_delta(clustered_matrix, labels, coloring, fs=(20, 15)):
  fig = plt.figure(figsize=fs)
  dn = dendrogram(clustered_matrix, labels=labels, orientation='left', color_threshold = 0)

  ax = plt.gca()
  ax.invert_yaxis()
  xlbls = ax.get_ymajorticklabels()
  for lbl in xlbls:
    color_decider = lbl.get_text()
    lbl.set_color(coloring[color_decider])
  plt.show()

In [ ]:
clustered, labels = clustering_delta(matrix)

In [ ]:
drawing_clustering_delta(clustered, labels, color_by_age, fs=(20, 40))

# Metrics

## ARI & NMI

In [ ]:
# we need only corpora/villages that are present in the subset

data_dict = dict()
villages_list = list()

for audio, village in file_corpora_dict.items():
  if audio in only_one_word:
    data_dict[audio] = village
    if village not in villages_list:
      villages_list.append(village)

data = list(data_dict.keys())
true_labels = list(data_dict.values())

# choosing a threshold for the linkage distance based on the number of clusters in the ground truth
t = len(villages_list)

# cutting a cluster tree - retrieving flat clustering and labels
labels = fcluster(clustered, t, criterion='maxclust')

if len(true_labels) != len(labels):
    raise ValueError('The number of samples in true_labels and labels are different')

# calculating the metrics between the true labels and the clustering labels
nmi = normalized_mutual_info_score(true_labels, labels)
ari = adjusted_rand_score(true_labels, labels)

print('These are metrics for the word "{}".'.format(the_word))
print(f"The NMI score is {nmi:.8f}")
print(f"The ARI score is {ari:.8f}")

In [ ]:
with open("{}_nmi.txt".format(the_word), "w") as nmi_file:
    nmi_file.write(str(nmi))

with open("{}_ari.txt".format(the_word), "w") as ari_file:
    ari_file.write(str(ari))

## Comparing geographical data with audio corpus data



In [ ]:
# that is where we upload a saved matrix from `comparison` function bact to the environment 

matrix = pd.read_pickle("orig.pkl")

In [ ]:
# retrieving coordinates 

tsv_path = '/content/drive/MyDrive/corpora_villages.tsv'
geo_df = pd.read_table(tsv_path)
only_needed_geo_info = geo_df[['name', 'lat', 'lon']]
only_needed_geo_info = only_needed_geo_info.drop(labels=[97], axis=0)

In [ ]:
meta_df = []
geo_df = []

In [ ]:
# this dictionary contains corpora's names in the dataset as keys and the names from geo.dataframe as values

dict_df_village_names = {'rogovatka':'Corpus of Rogovatka dialect',
 'malinino':'Corpus of the Russian dialect spoken in the village Malinino',
 'manturovo':'Corpus of the Russian dialect spoken in Manturovo',
 'lukhteza':'Corpus of Lukh and Teza river basins dialects',
 'teza':'Corpus of Lukh and Teza river basins dialects',
 'pyoza':'Corpus of the Russian dialect spoken in the villages of the Middle Pyoza',
 'luzhnikovo':'Luzhnikovo Corpus',
 'tserkovnoe':'Corpus of the Russian dialect spoken in Tserkovnoe',
 'don':'Corpus of the Russian dialect spoken in the villages of the Don river',
 'khislavichi':'Corpus of the Russian dialect spoken in Khislavichi district',
 'makeevo':'Corpus of Shetnevo and Makeevo',
 'nekhochi':'Corpus of the Russian dialect spoken in Nekhochi',
 'vyya':'Upper Pinega and Vyya Corpus',
 'keba':'Corpus of the Russian dialect spoken in Keba'}

In [ ]:
# thus fucntions calculates medians for each corpus' audio

def calculate_medians(matrix):
  df_for_median = matrix.apply(pd.to_numeric, errors='coerce')

  class_names = list(set([x.split('_')[-1] for x in df_for_median.index]))
  class_distances = {}

  for class_name in class_names:
      class_rows = [x for x in df_for_median.index if x.split('_')[-1] == class_name]
      class_distances[class_name] = df_for_median.loc[class_rows, class_rows].median().median()

  new_matrix = pd.DataFrame(columns=class_names, index=class_names)

  for i in range(len(class_names)):
      for j in range(len(class_names)):
          if i == j:
              new_matrix.iloc[i, j] = 0
          else:
              new_matrix.iloc[i, j] = class_distances[class_names[i]] + class_distances[class_names[j]]

  new_matrix -= df_for_median.median().median()

  new_matrix.values[[np.arange(len(class_names))]*2] = 0

  return class_names, new_matrix

In [ ]:
# this function compares the medians with the coordinates and calculates mantel test
# NB this function actually broke kernels - we used it just to get each and save it
# virtually, we used the code after this function to get the results of mantel test

def compare_geo_audio(comparison_matrix, dict_df_village_names = dict_df_village_names, coor_df = only_needed_geo_info):
  
  villages, village_medians = calculate_medians(comparison_matrix)

  list_of_villages_in_matrix = list()

  for village in villages:
    for file_name, df_name in dict_df_village_names.items():
      if village == file_name:
        list_of_villages_in_matrix.append(df_name)

  village_location_list = list()

  for one_village in list_of_villages_in_matrix:
  
    subdf_1row = coor_df.loc[coor_df.name == one_village]
    village_name = subdf_1row.name.iloc[0]

    lat = radians(subdf_1row.lat.iloc[0])
    lon = radians(subdf_1row.lon.iloc[0])
    
    village_location_list.append([lat, lon])

  geo_distance_matrix = haversine_distances(village_location_list)
  geo_distance_matrix = pd.DataFrame(geo_distance_matrix * 6371000/1000)

  village_medians_sqr_mtx, geo_sqr_mtx = squareform(village_medians), squareform(geo_distance_matrix) 
  
  mantel_test = mantel(village_medians_sqr_mtx, geo_sqr_mtx)

  return mantel_test
  #return geo_distance_matrix
  #return village_medians

We saved one matrix then restarted the kernel and save another one.
Then we restarted the kernel again and used the saved data to perform Mantel test

In [ ]:
# here we saved data

#compare_geo_audio(matrix).to_pickle("geo.pkl")
compare_geo_audio(matrix).to_pickle("vill.pkl")

In [ ]:
# here we read data

geo_distance = pd.read_pickle("geo.pkl")
vill_medians = pd.read_pickle("vill.pkl")

In [ ]:
# calculating mantel test

mantel_test = mantel(squareform(vill_medians), squareform(geo_distance))

In [ ]:
# saving the result in a file

with open("{}_mantel.txt".format('menja'), "w") as mantel_file:
    mantel_file.write(str(mantel_test))